In [30]:
import numpy as np
import pandas as pd
import scipy
import nltk
import sklearn
import random
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB

## reuters 21-578 modApte version
> a collection of 10,788 documents from the Reuters financial newswire service, partitioned into a training set with 7769 documents and a test set with 3019 documents

In [2]:
nltk.download('reuters')
nltk.download('punkt') # needed for tokenization

[nltk_data] Downloading package reuters to /home/felipe/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /home/felipe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dataset = nltk.corpus.reuters
dataset.root

ZipFilePathPointer(u'/home/felipe/nltk_data/corpora/reuters.zip', u'reuters/')

In [4]:
# dataset.readme()

In [5]:
len(dataset.categories())

90

In [6]:
len(dataset.fileids())

10788

In [7]:
fileids = dataset.fileids()
sample_fileid = [ fileids[i] for i in sorted(random.sample(xrange(len(fileids)), 1)) ][0]
sample_fileid

'training/11976'

In [8]:
dataset.abspath(sample_fileid)

ZipFilePathPointer(u'/home/felipe/nltk_data/corpora/reuters.zip', u'reuters/training/11976')

In [9]:
len(dataset.words(sample_fileid))

75

In [10]:
dataset.words(sample_fileid)

[u'JOHN', u'O', u'.', u'BUTLER', u'CO', u'&', u'lt', ...]

In [11]:
dataset.raw(sample_fileid)

u'JOHN O. BUTLER CO &lt;BUTC> 2ND QTR FEB 28 NET\n  Shr 19 cts vs 18 cts\n      Net 1,230,041 vs 1,153,280\n      Sales 10,909,729 vs 9,675,355\n      Six mths\n      Shr 31 cts vs 29 cts\n      Net 2,019,930 vs 1,857,357\n      Sales 21.0 mln vs 17.8 mln\n  \n\n'

In [12]:
dataset.words(sample_fileid)

[u'JOHN', u'O', u'.', u'BUTLER', u'CO', u'&', u'lt', ...]

In [13]:
dataset.sents(sample_fileid)

[[u'JOHN', u'O', u'.', u'BUTLER', u'CO', u'&', u'lt', u';', u'BUTC', u'>', u'2ND', u'QTR', u'FEB', u'28', u'NET', u'Shr', u'19', u'cts', u'vs', u'18', u'cts', u'Net', u'1', u',', u'230', u',', u'041', u'vs', u'1', u',', u'153', u',', u'280', u'Sales', u'10', u',', u'909', u',', u'729', u'vs', u'9', u',', u'675', u',', u'355', u'Six', u'mths', u'Shr', u'31', u'cts', u'vs', u'29', u'cts', u'Net', u'2', u',', u'019', u',', u'930', u'vs', u'1', u',', u'857', u',', u'357', u'Sales', u'21', u'.', u'0', u'mln', u'vs', u'17', u'.', u'8', u'mln']]

In [14]:
dataset.paras(sample_fileid)

[[[u'JOHN', u'O', u'.', u'BUTLER', u'CO', u'&', u'lt', u';', u'BUTC', u'>', u'2ND', u'QTR', u'FEB', u'28', u'NET', u'Shr', u'19', u'cts', u'vs', u'18', u'cts', u'Net', u'1', u',', u'230', u',', u'041', u'vs', u'1', u',', u'153', u',', u'280', u'Sales', u'10', u',', u'909', u',', u'729', u'vs', u'9', u',', u'675', u',', u'355', u'Six', u'mths', u'Shr', u'31', u'cts', u'vs', u'29', u'cts', u'Net', u'2', u',', u'019', u',', u'930', u'vs', u'1', u',', u'857', u',', u'357', u'Sales', u'21', u'.', u'0', u'mln', u'vs', u'17', u'.', u'8', u'mln']]]

In [15]:
# http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
corpus_train = []
corpus_test = []
for fileid in dataset.fileids():
    document = dataset.raw(fileid)
    if re.match('training/',fileid):
        corpus_train.append(document)
    else:
        corpus_test.append(document)

In [16]:
len(corpus_train),len(corpus_test)

(7769, 3019)

In [17]:
def preprocessor(string):
    repl = re.sub('&lt;','',string)
    return repl.lower()

In [18]:
vectorizer = CountVectorizer(
                min_df=10, # tweaking this parameter reduces the length of the feature vector
                strip_accents='ascii',
                preprocessor=preprocessor,
                stop_words='english')

In [19]:
# need to use both corpuses for fitting because otherwise there may be words that only occur in the
# training set or in the test set
full_corpus = corpus_train + corpus_test
vectorizer.fit(full_corpus)

X_train_counts = vectorizer.transform(corpus_train)
X_test_counts = vectorizer.transform(corpus_test)
X_full_counts = vectorizer.transform(full_corpus)

X_train_counts.shape,X_test_counts.shape, X_full_counts.shape

((7769, 6462), (3019, 6462), (10788, 6462))

In [20]:
#uncomment these to see how the vectorizer is analyzing, tokenizing and preprocessing documents

#vectorizer.build_analyzer()(dataset.raw(fileid))
#vectorizer.build_tokenizer()("ADVANCED INSTITUTIONAL &lt;AIMS> CUTS WORKFORCE\n  Advanced Institutional ")
#vectorizer.build_preprocessor()("ADVANCED INSTITUTIONAL &lt;AIMS> CUTS WORKFORCE\n  Advanced Institutional ")

In [21]:
X_train_counts[0].toarray().ravel()

array([0, 0, 0, ..., 1, 0, 0])

In [22]:
X_test_counts[0].toarray().ravel()

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
transformer = TfidfTransformer()
# again, we need to fit the transformer to all documents (train and test)
transformer.fit(X_full_counts)

X_train_tfidf = transformer.transform(X_train_counts)
X_test_tfidf = transformer.transform(X_test_counts)
X_full_tfidf = transformer.transform(X_full_counts)

X_train_tfidf.shape, X_test_tfidf.shape, X_full_tfidf.shape

((7769, 6462), (3019, 6462), (10788, 6462))

In [24]:
X_train_tfidf[0].toarray().ravel()

array([ 0.       ,  0.       ,  0.       , ...,  0.0466051,  0.       ,  0.       ])

In [25]:
X_test_tfidf[0].toarray().ravel()

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [26]:
Y_train = []
Y_test = []

for (idx,fileid) in enumerate(dataset.fileids()):    
    categories = '*'.join(dataset.categories(fileid))

    if re.match('training/',fileid):
        Y_train.append(categories)
    else:
        Y_test.append(categories)

series_train = pd.Series(Y_train)
Y_train_df = series_train.str.get_dummies(sep='*')

series_test = pd.Series(Y_test)
Y_test_df = series_test.str.get_dummies(sep='*')

Y_train = Y_train_df.values
Y_test = Y_test_df.values

Y_train.shape,Y_test.shape

((7769, 90), (3019, 90))

In [33]:
clf = GaussianNB()
meta_clf = OneVsRestClassifier(clf)

meta_clf.fit(X_train_tfidf.toarray(),Y_train)


OneVsRestClassifier(estimator=GaussianNB(), n_jobs=1)

In [35]:
Y_pred = meta_clf.predict(X_test_tfidf.toarray())

In [36]:
f1_score(Y_test,Y_pred,average='micro')

0.5320712065555242